# 녹음 -> blob에 저장 -> 로드 -> STT -> blob에 저장 연결 코드

In [8]:
import wave
import pyaudio
import azure.cognitiveservices.speech as speechsdk
from azure.storage.blob import BlobServiceClient
from datetime import datetime
import io
import keyboard
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# Azure Storage 연결 문자열을 환경 변수에서 가져오기
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
BLOB_AUDIO_CONTAINER = "audio-files"
BLOB_TEXT_CONTAINER = "stt-results"

# 녹음 설정 (STT 최적화)
FORMAT = pyaudio.paInt16  # PCM 16-bit format
CHANNELS = 1  # Mono
RATE = 16000  # 16kHz (Azure STT 권장)
CHUNK = 1024  # Buffer size

def create_container_if_not_exists(container_name):
    """Azure Blob Storage 컨테이너가 없으면 생성"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    container_client = blob_service_client.get_container_client(container_name)
    if not container_client.exists():
        container_client.create_container()

create_container_if_not_exists(BLOB_AUDIO_CONTAINER)
create_container_if_not_exists(BLOB_TEXT_CONTAINER)

def record_and_upload():
    """마이크로 음성을 녹음한 후, WAV 파일로 변환하여 Azure Blob Storage에 저장"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    blob_name = f"live_audio_{timestamp}.wav"
    local_audio_path = f"./{blob_name}"
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK, input_device_index=0)

    print("실시간 녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)")
    frames = []

    try:
        while True:
            data = stream.read(CHUNK)
            frames.append(data)
            if keyboard.is_pressed("enter"):
                print("녹음 종료")
                break
    except KeyboardInterrupt:
        print("녹음 종료 (CTRL+C 입력 감지됨)")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    # WAV 파일로 저장
    with wave.open(local_audio_path, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))

    print(f"WAV 파일 저장 완료: {local_audio_path}")

    # Azure Blob Storage에 업로드
    with open(local_audio_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)

    print(f"오디오 파일 업로드 완료: {blob_name}")
    return blob_name

class BinaryAudioStreamReader(speechsdk.audio.PullAudioInputStreamCallback):
    def __init__(self, audio_stream):
        super().__init__()
        self.audio_stream = io.BytesIO(audio_stream)

    def read(self, buffer: memoryview) -> int:
        size = len(buffer)
        data = self.audio_stream.read(size)
        buffer[:len(data)] = data
        return len(data)

    def close(self):
        self.audio_stream.close()

def transcribe_audio_from_blob_direct(blob_name):
    """Azure STT API를 사용하여 Blob Storage에서 직접 변환"""
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    blob_client = blob_service_client.get_blob_client(container=BLOB_AUDIO_CONTAINER, blob=blob_name)
    blob_data = blob_client.download_blob().readall()

    speech_key = os.getenv("SPEECH_API_KEY")
    speech_region = os.getenv("SPEECH_REGION")

    if not speech_key or not speech_region:
        raise ValueError("환경 변수 SPEECH_API_KEY 또는 SPEECH_REGION이 설정되지 않았습니다.")

    speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=speech_region)
    speech_config.speech_recognition_language = "ko-KR"  # 한국어 설정 추가

    stream_reader = BinaryAudioStreamReader(blob_data)
    audio_input_stream = speechsdk.audio.PullAudioInputStream(stream_reader)
    audio_config = speechsdk.audio.AudioConfig(stream=audio_input_stream)

    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print("음성을 텍스트로 변환 중...")
    result = speech_recognizer.recognize_once()

    if result.reason == speechsdk.ResultReason.RecognizedSpeech:
        print(f"STT 변환 결과: {result.text}")
        return result.text
    else:
        print("음성 인식 실패")
        return None

def upload_stt_result_to_blob(text_data, audio_blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(BLOB_CONNECTION_STRING)
    text_blob_name = audio_blob_name.replace(".wav", ".txt")
    blob_client = blob_service_client.get_blob_client(container=BLOB_TEXT_CONTAINER, blob=text_blob_name)
    blob_client.upload_blob(text_data, overwrite=True)
    print(f"STT 변환된 텍스트 업로드 완료: {text_blob_name}")

blob_audio_name = record_and_upload()
stt_text = transcribe_audio_from_blob_direct(blob_audio_name)

if stt_text:
    upload_stt_result_to_blob(stt_text, blob_audio_name)


실시간 녹음 시작... (Enter 또는 CTRL+C를 눌러 종료)
녹음 종료
WAV 파일 저장 완료: ./live_audio_20250203_202305.wav
오디오 파일 업로드 완료: live_audio_20250203_202305.wav
음성을 텍스트로 변환 중...
STT 변환 결과: 안녕하세요. 오늘 하시는 수술은 복강경 담당 절제술입니다. 환자분은 담낭 용종이 있어서 수술을 받으시는 건데요. 우선 담낭 용종이랑 먼지 간단히 설명해 드릴게요.
STT 변환된 텍스트 업로드 완료: live_audio_20250203_202305.txt
